In [ ]:
%matplotlib qt
import matplotlib.pyplot as plt

import os.path as op
import numpy as np
import mne

mne.set_log_level('WARNING')

subjects_dir = op.expanduser("~/data/pic-name-data-bids/MRI/")
res_dir = op.expanduser("~/research/results/pic_name")

In [ ]:
bili = [8, 9]
# clil = [1, 6, 10, 11, 15]
mono = [3, 4, 5, 13, 14]
subjects = ["sub-%02d" % i for i in (bili + mono)]
# subjects = ["sub-%02d" % i for i in bili]
subjects

In [ ]:
evokeds_list = []
for subject in subjects:
    fname = op.join(res_dir, subject, subject+'-ave.fif')
    ev = mne.read_evokeds(fname)
    evokeds_list.append(ev[0])

In [ ]:
f, axes = plt.subplots(2, 4, figsize=(13, 9), sharex=True, sharey=True)
for ax, subject, evokeds in zip(axes.ravel(), subjects, evokeds_list):
    mne.viz.plot_evoked(evokeds, picks='MEG2343', show=False, axes=ax, 
                        titles=subject, spatial_colors=True, xlim=(-0.1,0.6))  
plt.tight_layout()

# plt.savefig(op.join(res_dir, 'evoked_all_subjects.pdf'))

In [ ]:
evoked_gave = mne.grand_average(evokeds_list)
evoked_gave.plot(spatial_colors=True)

# plt.savefig(op.join(res_dir, 'evoked_group.pdf'))

In [ ]:
# evkd = []
# for subject, evokeds in zip(subjects, evokeds_list):
#     mne.viz.plot_evoked(evokeds, picks='MEG2343', show=False, xlim=(-0.1,0.6))
# plt.show()

In [ ]:
surfer_kwargs = dict(subject="fsaverage",
                     background="white", foreground='black',
                     cortex=("gray", -1, 6, True), smoothing_steps=10,
                     clim=dict(kind='value', lims=[2.5, 5, 6]),
                     time_viewer=False)

In [ ]:
res_dir = op.expanduser("~/research/results/pic_name")
stc_fnames = []
for subject in subjects:
    stc_fnames.append(op.join(res_dir, subject + '/dspm_%s' % subject))
stcs = [mne.read_source_estimate(stc_fname) for stc_fname in stc_fnames]
average_stc = np.mean(stcs)

In [ ]:
brain = average_stc.plot(initial_time=0.36, subjects_dir=subjects_dir, hemi="both", views='dor',
                         **surfer_kwargs)

# brain.save_image(op.join(res_dir, "dspm_fsavg_t=400ms.png"))

In [ ]:
labels = mne.read_labels_from_annot('fsaverage', 'HCPMMP1', 'both', subjects_dir=subjects_dir)
labels_vis = []
ROI = ['L_V1_ROI-lh', 'R_V1_ROI-rh']
for lbl in labels:
    if lbl.name in ROI:
        labels_vis.append(lbl)
label = labels_vis[0]
for i in range(1, len(labels_vis)):
    label = label + labels_vis[i]

average_stc_label = average_stc.in_label(label)
average_stc_label_mean = np.mean(average_stc_label.data, axis=0)

In [ ]:
times = 1e3 * average_stc.times # times in ms
plt.figure()
h0 = plt.plot(times, average_stc_label.data.T, 'k')
h1, = plt.plot(times, average_stc_label_mean, 'r', linewidth=3)
plt.show()

# plt.savefig(op.join(res_dir, 'evoked_label_group.pdf'))